<a href="https://colab.research.google.com/github/shelestmariia/Online-Store-Exploratory-Data-Analysis/blob/main/A_B_testing_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-bigquery
from google.colab import auth
from google.cloud import bigquery
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.38.0
    Uninstalling google-cloud-bigquery-3.38.0:
      Successfully uninstalled google-cloud-bigquery-3.38.0


In [ ]:
query = """
WITH
  session_info AS (
    SELECT
      s.date,
      s.ga_session_id,
      sp.country,
      sp.device,
      sp.continent,
      sp.channel,
      ab.test,
      ab.test_group
    FROM `DA.ab_test` ab
    JOIN `DA.session` s
      ON ab.ga_session_id = s.ga_session_id
    JOIN `DA.session_params` sp
      ON sp.ga_session_id = ab.ga_session_id
  ),
  session_with_orders AS (
    SELECT
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      COUNT(DISTINCT o.ga_session_id) AS session_with_orders
    FROM `DA.order` o
    JOIN session_info
      ON o.ga_session_id = session_info.ga_session_id
    GROUP BY
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group
  ),
  events AS (
    SELECT
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      sp.event_name,
      COUNT(sp.ga_session_id) AS event_cnt
    FROM `DA.event_params` sp
    JOIN session_info
      ON sp.ga_session_id = session_info.ga_session_id
    GROUP BY
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      sp.event_name
  ),
  session AS (
    SELECT
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
    FROM session_info
    GROUP BY
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group
  ),
  account AS (
    SELECT
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group,
      COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
    FROM `DA.account_session` acs
    JOIN session_info
      ON acs.ga_session_id = session_info.ga_session_id
    GROUP BY
      session_info.date,
      session_info.country,
      session_info.device,
      session_info.continent,
      session_info.channel,
      session_info.test,
      session_info.test_group
  )
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  'session with orders' AS event_name,
  session_with_orders.session_with_orders AS value
FROM session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt AS value
FROM events
UNION ALL
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  'session' AS event_name,
  session_cnt AS value
FROM session
UNION ALL
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  'new account' AS event_name,
  new_account_cnt AS value
FROM account;
"""

In [ ]:
query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-12-08,Palestine,desktop,Asia,Direct,4,2,new account,1
1,2020-12-08,Palestine,desktop,Asia,Direct,3,2,new account,1
2,2020-11-06,Puerto Rico,desktop,Americas,Social Search,2,2,new account,1
3,2020-11-06,Puerto Rico,desktop,Americas,Social Search,1,1,new account,1
4,2020-12-08,Croatia,desktop,Europe,Direct,4,2,new account,1


In [ ]:
df["event_name"] = df["event_name"].replace("new account", "new_account")

In [ ]:
grouped_df = (df.groupby(["test", "test_group", "event_name"], as_index=False).agg(total_value=("value", "sum")))

grouped_df

,test,test_group,event_name,total_value
0,1,1,add_payment_info,1988
1,1,1,add_shipping_info,3034
2,1,1,add_to_cart,1395
3,1,1,begin_checkout,3784
4,1,1,click,368
...,...,...,...,...
147,4,2,user_engagement,273633
148,4,2,view_item,98928
149,4,2,view_item_list,6
150,4,2,view_promotion,51985


In [ ]:
pivot_df = (grouped_df.pivot(index=["test", "event_name"], columns="test_group", values="total_value").fillna(0).reset_index())
pivot_df.columns.name = None
pivot_df

,test,event_name,1,2
0,1,add_payment_info,1988,2229
1,1,add_shipping_info,3034,3221
2,1,add_to_cart,1395,1366
3,1,begin_checkout,3784,4021
4,1,click,368,353
...,...,...,...,...
71,4,user_engagement,273946,273633
72,4,view_item,98619,98928
73,4,view_item_list,5,6
74,4,view_promotion,52672,51985


In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# list of metrics
metrics = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new_account"]

# aggregate events by test and test_group
agg = (df[df["test_group"].isin([1, 2])].groupby(["test", "test_group", "event_name"], as_index=False).agg(events=("value", "sum")))

# highlight sessions
sessions = (agg[agg["event_name"] == "session"].rename(columns={"events": "denominator"}).drop(columns="event_name"))

# choosing metrics
metrics_df = agg[agg["event_name"].isin(metrics)].rename(columns={"events": "numerator"})

# add sessions as the denominator
metrics_df = metrics_df.merge(sessions, on=["test", "test_group"], how="left")

# CR calculation
metrics_df["conversion_rate"] = metrics_df["numerator"] / metrics_df["denominator"]

# wide-format: control (cd) vs variant (vr)
wide = metrics_df.pivot(
    index=["test", "event_name"],
    columns="test_group",
    values=["numerator", "denominator", "conversion_rate"])

# renaming columns
wide.columns = [
    f"{col[0]}_{'cd' if col[1]==1 else 'vr'}"
    for col in wide.columns]

wide = wide.reset_index()

# add a metric column (event / session)
wide["metric"] = wide["event_name"] + " / session"
wide["numerator_event_name"] = wide["event_name"]
wide["denominator_event_name"] = "session"

# Z-test + lift for each metric
results = []

for _, row in wide.iterrows():
    count = np.array([row["numerator_cd"], row["numerator_vr"]])
    nobs = np.array([row["denominator_cd"], row["denominator_vr"]])

    z_stat, p_value = proportions_ztest(count, nobs)

    metric_change = ((row["conversion_rate_vr"] - row["conversion_rate_cd"]) / row["conversion_rate_cd"] * 100)

    results.append({
        "test_number": row["test"],
        "metric": row["metric"],
        "numerator_event_name": row["numerator_event_name"],
        "denominator_event_name": row["denominator_event_name"],
        "numerator_vr": row["numerator_vr"],
        "denominator_vr": row["denominator_vr"],
        "conversion_rate_vr": row["conversion_rate_vr"],
        "numerator_cd": row["numerator_cd"],
        "denominator_cd": row["denominator_cd"],
        "conversion_rate_cd": row["conversion_rate_cd"],
        "metric_change": metric_change,
        "z_stat": z_stat,
        "p_value": p_value,
        "significant": p_value < 0.05
    })

# final table
final_df = pd.DataFrame(results)

final_df

,test_number,metric,numerator_event_name,denominator_event_name,numerator_vr,denominator_vr,conversion_rate_vr,numerator_cd,denominator_cd,conversion_rate_cd,metric_change,z_stat,p_value,significant
0,1,add_payment_info / session,add_payment_info,session,2229,45193,0.049322,1988,45362,0.043825,12.542021,-3.924884,0.000087,True
1,1,add_shipping_info / session,add_shipping_info,session,3221,45193,0.071272,3034,45362,0.066884,6.560481,-2.603571,0.009226,True
2,1,begin_checkout / session,begin_checkout,session,4021,45193,0.088974,3784,45362,0.083418,6.660587,-2.978783,0.002894,True
3,1,new_account / session,new_account,session,3681,45193,0.081451,3823,45362,0.084278,-3.354299,1.542883,0.122859,False
4,2,add_payment_info / session,add_payment_info,session,2409,50244,0.047946,2344,50637,0.046290,3.576911,-1.240994,0.214608,False
5,2,add_shipping_info / session,add_shipping_info,session,3510,50244,0.069859,3480,50637,0.068724,1.650995,-0.709557,0.477979,False
6,2,begin_checkout / session,begin_checkout,session,4313,50244,0.085841,4262,50637,0.084168,1.988164,-0.952898,0.340642,False
7,2,new_account / session,new_account,session,4184,50244,0.083274,4165,50637,0.082252,1.241934,-0.588793,0.556000,False
8,3,add_payment_info / session,add_payment_info,session,3697,70439,0.052485,3623,70047,0.051722,1.474630,-0.643172,0.520112,False
9,3,add_shipping_info / session,add_shipping_info,session,5188,70439,0.073652,5298,70047,0.075635,-2.621211,1.413727,0.157442,False


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
final_df.to_csv("/content/drive/MyDrive/data.csv", index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Data file and Dashboards**

* [Data](https://drive.google.com/file/d/1twJGBz6bAY9cBVXTvY5Id0At1CE4eVkX/view?usp=sharing)
* [AB test](https://public.tableau.com/app/profile/mariia.shelest/viz/ABtest_17646009042150/ABtest?publish=yes)
* [Significance](https://public.tableau.com/app/profile/mariia.shelest/viz/Significance_17659929768420/Dashboard1?publish=yes)

# **Conclusion**

After analyzing the conducted A/B tests, it can be noted that **Test 1** was the most successful. The conversion rate increased for all metrics except *new_accounts / session*. The positive results were statistically significant, while the reduction in *new_accounts / session* was not statistically significant.

For **Test 2**, the increase in conversion was not statistically significant for any of the analyzed metrics.

In **Test 3**, the changes were neither substantial nor statistically significant, except for *begin_checkout / session*, which experienced a statistically significant reduction.

In **Test 4**, all metrics declined; for *begin_checkout / session* and *new_accounts / session*, the reductions were statistically significant.